# How to learn feature for functional maps

In this notebook, we show how to use deep functional maps to learn feature for 3d shape matching.

In [1]:
import os

os.environ["GEOMSTATS_BACKEND"] = "pytorch"

import torch
from torch.utils.data import DataLoader, random_split

from geomfum.dataset.torch import PairsDataset, ShapeDataset
from geomfum.descriptor.learned import FeatureExtractor
from geomfum.forward_functional_map import ForwardFunctionalMap
from geomfum.learning.losses import (
    BijectivityLoss,
    GeodesicError,
    LaplacianCommutativityLoss,
    LossManager,
    OrthonormalityLoss,
)
from geomfum.learning.models import FMNet
from geomfum.learning.trainer import DeepFunctionalMapTrainer


First, we define our model. We can instantiate it combining feature extractors and forward logic, however, we provide some classic frameworks, like FMNet.

In [ ]:
# Build the model
fmap_module = ForwardFunctionalMap(1e3, 1, True)

feature_extractor = FeatureExtractor.from_registry(
    which="diffusionnet",
    cache_dir="../../../datasets/faust/train_set/diffusion",
    device="cuda",
)

fumctional_map_model = FMNet(
    feature_extractor=feature_extractor,
    fmap_module=fmap_module,
)


Then, we instantiate the training dataset and we split it for training purposes.

In [3]:
TRAIN_SET_PATH = "../../../datasets/faust/train_set/"
dataset = ShapeDataset(
    TRAIN_SET_PATH, spectral=True, distances=False, device="cuda", k=30
)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_shapes, validation_shapes = random_split(dataset, [train_size, val_size])
train_dataset = PairsDataset(
    train_shapes,
    pair_mode="all",
)
train_loader = DataLoader(
    train_dataset, batch_size=1, shuffle=True
)  # if shapes have different number of vertices, set batch_size=1

validation_dataset = PairsDataset(
    validation_shapes,
    pair_mode="all",
)
val_loader = DataLoader(validation_dataset, batch_size=1, shuffle=True)

C:\Users\giuli\OneDrive\Research\geomfum_proj\geomfum\geomfum\_backend\pytorch\sparse.py:22: UserWarning: Sparse CSC tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  return _torch.sparse_csc_tensor(ccol_indices, row_indices, values, size=array.shape)


Since a lot of time we do not need to store distances for training shapes, but they are usefull for validation shapes, we can do the following trick

In [17]:
# build the train and test loaders
TRAIN_SET_PATH = "../../../datasets/faust/train_set/"
# instantiate the full dataset with default attributes
full_dataset = ShapeDataset(TRAIN_SET_PATH, device="cpu")
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_indices, val_indices = torch.utils.data.random_split(
    range(len(full_dataset)), [train_size, val_size]
)

# create train and validation datasets with different attributes
train_shapes = ShapeDataset(
    TRAIN_SET_PATH,
    spectral=True,
    distances=False,
    device="cuda",
    k=30,
    indices=train_indices,
)
validation_shapes = ShapeDataset(
    TRAIN_SET_PATH,
    spectral=True,
    distances=True,
    device="cuda",
    k=30,
    indices=val_indices,
)
# kwe create pairs
train_dataset = PairsDataset(
    train_shapes,
    pair_mode="all",
    n_pairs=100,
)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
validation_dataset = PairsDataset(
    validation_shapes,
    pair_mode="all",
    n_pairs=100,
)
val_loader = DataLoader(validation_dataset, batch_size=1, shuffle=True)

we build optimizer

In [20]:
optimizer = torch.optim.Adam(fumctional_map_model.parameters(), lr=1e-3)

Now we define the losses that we will cnsider. Again we can define our own losses, howver we provide some classic functional map energies, like the orthonormality loss.

In [21]:
# define the loss
losses = [
    OrthonormalityLoss(weight=1.0),
    BijectivityLoss(weight=1.0),
    LaplacianCommutativityLoss(weight=1e-3),
]
loss_manager = LossManager(losses)

losses = [
    GeodesicError(),
]

val_loss_manager = LossManager(losses)

We have defined a trainer for simplicity that thakes as input model, losses, train and val datasets and optimizer and manages the training loops.

In [22]:
trainer = DeepFunctionalMapTrainer(
    model=fumctional_map_model,
    train_loss_manager=loss_manager,
    val_loss_manager=val_loss_manager,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device="cuda",
)

In [23]:
trainer.train()

INFO: Epoch [1/100] - Training
Epoch 1/100 (Train):  13%|█▎        | 510/4032 [03:55<27:05,  2.17batch/s, Loss=66.7191] 


KeyboardInterrupt: 